In [1]:
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

site = 'https://www.amazon.co.za' #amazon.co.za
headers = ['Name', 'Price', 'Rating', 'Rating Count', 'URL Link']
df = pd.DataFrame(columns = headers)

In [2]:
url = site + '/s?k=samsung&ref=nb_sb_noss' #Site Extension
response = requests.get(url)
soup2 = BeautifulSoup(response.content, 'html.parser')
soup = BeautifulSoup(soup2.prettify(), 'html.parser')
dontstop = True
counter = 0

while dontstop:
    x = soup.find_all('div', class_ = "s-asin")
    count = len(x)
    for i in range(count):
        data = []
    #DATA
        x = soup.find_all('div', class_ = "s-asin")
        items = x[i]
    #NAME
        name = items.find('h2')#.text.strip()
        if name is None:
            name = 'N/A'
        else:
            name = name.text.strip()
        name = str(name)
        if ',' in name:
            name = name.replace(',', ';')
        #print('Name: ', name)
        data.append(name)
    #PRICE
        a = items.find('span', class_ = "a-price")
        '''if a is None:
            a = '0,000.00'
        else:
            a = a.text'''
        if a is None:
            a = items.find('span', class_ = "a-color-base")
            if a is None:
                a = '0,000.00'
            else:
                a = a.text
        else:
            a = a.text
        b = re.findall(r'\d{1,3},?\d{1,3}\.\d{2}', a)
        c = b[0].replace(",", "")
        price = float(c)
        '''if price == 0:
            price = 'N/A'
        else:
            price = "{:.2f}".format(price)'''
        #print('Price: ', price)
        #price = "{:.2f}".format(price)
        data.append(price)
    #RATING
        rating = items.find('span', class_ = "a-icon-alt")#.text.strip()
        if rating is None:
            rating = 'N/A'
        else:
            rating = rating.text.strip()
        rating = str(rating)
        #print('Rating: ', rating)
        for x in rating:
            if x == ',':
                x = ';'
        data.append(rating)
        #RATING_COUNT
        rating_c = items.find('span', class_ = "a-size-base")#.text.strip()
        if rating_c is None:
            rating_c = 'N/A'
        else:
            rating_c = rating_c.text.strip()
            try:
                rating_c = int(rating_c)
            except:
                rating_c = 'N/A'
        rating_c = str(rating_c)
        #print('Rating Count: ', rating_c)
        for x in rating_c:
            if x == ',':
                x = ';'
        data.append(rating_c)
    #URL
        u = items.find('a', href = True)
        if (u == None) or ('#customerReviews' in u) or ('javascript:void' in u):
            item_url = 'N/A'
        else:
            item_url = u['href']
            item_url = site + item_url
        item_url = str(item_url)
        #print('Site: ', item_url)
        for x in item_url:
            if x == ',':
                x = ';'
        data.append(item_url)
        counter = counter + 1
        #print(counter)
        #print('')
    #ABOUT
        '''item_response = requests.get(item_url)
        item_soup2 = BeautifulSoup(item_response.content, 'html.parser')
        item_soup = BeautifulSoup(item_soup2.prettify(), 'html.parser')
        about_item = ""
        item_a = item_soup.find_all('li', class_ = "a-spacing-mini")
        about_count = 0
        if item_a == None:
            about_item = 'N/A'
        else:
            for abouts in item_a:
                about_item = about_item + abouts.text.strip()
                about_count = about_count + 1
                if about_count < len(item_a):
                    about_item = about_item + "; "
        data.append(about_item)''' # TOO LONG TO RUN EACH ITEM
        length = len(df)
        df.loc[length] = data
        
    n = soup.find('a', class_ = "s-pagination-next")
    if n == None:
        dontstop = False
    else:
        nxt = n['href']
        nxt = site + nxt
        url = nxt
        response = requests.get(url)
        soup2 = BeautifulSoup(response.content, 'html.parser')
        soup = BeautifulSoup(soup2.prettify(), 'html.parser')

df

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22679.00,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
4,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
...,...,...,...,...,...
301,Body Glove A35 5G/A55 5G Tempered Glass Screen...,191.04,N/A,N/A,https://www.amazon.co.za/Body-Glove-Tempered-c...
302,Samsung Mobile Case for Galaxy Z Fold5 Series;...,799.00,4.0 out of 5 stars,75,https://www.amazon.co.za/Samsung-Mobile-Galaxy...
303,Compatible To Samsung Z Fold5 phone case with ...,399.00,N/A,N/A,https://www.amazon.co.za/Compatible-Samsung-fo...
304,Screen Protector For Samsung Galaxy A35/55 - T...,147.00,N/A,N/A,https://www.amazon.co.za/Screen-Protector-Sams...


In [3]:
url = site + '/s?k=huawei&ref=nb_sb_noss' #Site Extension
response = requests.get(url)
soup2 = BeautifulSoup(response.content, 'html.parser')
soup = BeautifulSoup(soup2.prettify(), 'html.parser')
dontstop = True
counter = 0

while dontstop:
    x = soup.find_all('div', class_ = "s-asin")
    count = len(x)
    for i in range(count):
        data = []
    #DATA
        x = soup.find_all('div', class_ = "s-asin")
        items = x[i]
    #NAME
        name = items.find('h2')#.text.strip()
        if name is None:
            name = 'N/A'
        else:
            name = name.text.strip()
        name = str(name)
        if ',' in name:
            name = name.replace(',', ';')
        #print('Name: ', name)
        data.append(name)
    #PRICE
        a = items.find('span', class_ = "a-price")
        '''if a is None:
            a = '0,000.00'
        else:
            a = a.text'''
        if a is None:
            a = items.find('span', class_ = "a-color-base")
            if a is None:
                a = '0,000.00'
            else:
                a = a.text
        else:
            a = a.text
        b = re.findall(r'\d{1,3},?\d{1,3}\.\d{2}', a)
        c = b[0].replace(",", "")
        price = float(c)
        '''if price == 0:
            price = 'N/A'
        else:
            price = "{:.2f}".format(price)'''
        #print('Price: ', price)
        #price = "{:.2f}".format(price)
        data.append(price)
    #RATING
        rating = items.find('span', class_ = "a-icon-alt")#.text.strip()
        if rating is None:
            rating = 'N/A'
        else:
            rating = rating.text.strip()
        rating = str(rating)
        #print('Rating: ', rating)
        for x in rating:
            if x == ',':
                x = ';'
        data.append(rating)
    #RATING_COUNT
        rating_c = items.find('span', class_ = "a-size-base")#.text.strip()
        if rating_c is None:
            rating_c = 'N/A'
        else:
            rating_c = rating_c.text.strip()
            try:
                rating_c = int(rating_c)
            except:
                rating_c = 'N/A'
        rating_c = str(rating_c)
        #print('Rating Count: ', rating_c)
        for x in rating_c:
            if x == ',':
                x = ';'
        data.append(rating_c)
    #URL
        u = items.find('a', href = True)
        if (u == None) or ('#customerReviews' in u) or ('javascript:void' in u):
            item_url = 'N/A'
        else:
            item_url = u['href']
            item_url = site + item_url
        item_url = str(item_url)
        #print('Site: ', item_url)
        for x in item_url:
            if x == ',':
                x = ';'
        data.append(item_url)
        counter = counter + 1
        #print(counter)
        #print('')
    #ABOUT
        '''item_response = requests.get(item_url)
        item_soup2 = BeautifulSoup(item_response.content, 'html.parser')
        item_soup = BeautifulSoup(item_soup2.prettify(), 'html.parser')
        about_item = ""
        item_a = item_soup.find_all('li', class_ = "a-spacing-mini")
        about_count = 0
        if item_a == None:
            about_item = 'N/A'
        else:
            for abouts in item_a:
                about_item = about_item + abouts.text.strip()
                about_count = about_count + 1
                if about_count < len(item_a):
                    about_item = about_item + "; "
        data.append(about_item)''' # TOO LONG TO RUN EACH ITEM
        length = len(df)
        df.loc[length] = data
        
    n = soup.find('a', class_ = "s-pagination-next")
    if n == None:
        dontstop = False
    else:
        nxt = n['href']
        nxt = site + nxt
        url = nxt
        response = requests.get(url)
        soup2 = BeautifulSoup(response.content, 'html.parser')
        soup = BeautifulSoup(soup2.prettify(), 'html.parser')

df

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22679.00,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
4,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
...,...,...,...,...,...
607,Body Glove Black Trim Border 3D Tempered Glass...,198.82,N/A,N/A,https://www.amazon.co.za/Body-Glove-Border-Tem...
608,Global Tech Parts Replacement LCD For Huawei P...,499.99,N/A,N/A,https://www.amazon.co.za/Replacement-Huawei-Sm...
609,Body Glove Black Trim Border Tempered Glass Sc...,152.68,N/A,N/A,https://www.amazon.co.za/Body-Glove-Border-Tem...
610,Funki Fox Huawei P60 PRO Soft & Smooth Phone C...,199.00,N/A,N/A,https://www.amazon.co.za/Funki-Fox-Huawei-Smoo...


In [4]:
url = site + '/s?k=sony&ref=nb_sb_noss' #Site Extension
response = requests.get(url)
soup2 = BeautifulSoup(response.content, 'html.parser')
soup = BeautifulSoup(soup2.prettify(), 'html.parser')
dontstop = True
counter = 0

while dontstop:
    x = soup.find_all('div', class_ = "s-asin")
    count = len(x)
    for i in range(count):
        data = []
    #DATA
        x = soup.find_all('div', class_ = "s-asin")
        items = x[i]
    #NAME
        name = items.find('h2')#.text.strip()
        if name is None:
            name = 'N/A'
        else:
            name = name.text.strip()
        name = str(name)
        if ',' in name:
            name = name.replace(',', ';')
        #print('Name: ', name)
        data.append(name)
    #PRICE
        a = items.find('span', class_ = "a-price")
        '''if a is None:
            a = '0,000.00'
        else:
            a = a.text'''
        if a is None:
            a = items.find('span', class_ = "a-color-base")
            if a is None:
                a = '0,000.00'
            else:
                a = a.text
        else:
            a = a.text
        b = re.findall(r'\d{1,3},?\d{1,3}\.\d{2}', a)
        c = b[0].replace(",", "")
        price = float(c)
        '''if price == 0:
            price = 'N/A'
        else:
            price = "{:.2f}".format(price)'''
        #print('Price: ', price)
        #price = "{:.2f}".format(price)
        data.append(price)
    #RATING
        rating = items.find('span', class_ = "a-icon-alt")#.text.strip()
        if rating is None:
            rating = 'N/A'
        else:
            rating = rating.text.strip()
        rating = str(rating)
        #print('Rating: ', rating)
        for x in rating:
            if x == ',':
                x = ';'
        data.append(rating)
    #RATING_COUNT
        rating_c = items.find('span', class_ = "a-size-base")#.text.strip()
        if rating_c is None:
            rating_c = 'N/A'
        else:
            rating_c = rating_c.text.strip()
            try:
                rating_c = int(rating_c)
            except:
                rating_c = 'N/A'
        rating_c = str(rating_c)
        #print('Rating Count: ', rating_c)
        for x in rating_c:
            if x == ',':
                x = ';'
        data.append(rating_c)
    #URL
        u = items.find('a', href = True)
        if (u == None) or ('#customerReviews' in u) or ('javascript:void' in u):
            item_url = 'N/A'
        else:
            item_url = u['href']
            item_url = site + item_url
        item_url = str(item_url)
        #print('Site: ', item_url)
        for x in item_url:
            if x == ',':
                x = ';'
        data.append(item_url)
        counter = counter + 1
        #print(counter)
        #print('')
    #ABOUT
        '''item_response = requests.get(item_url)
        item_soup2 = BeautifulSoup(item_response.content, 'html.parser')
        item_soup = BeautifulSoup(item_soup2.prettify(), 'html.parser')
        about_item = ""
        item_a = item_soup.find_all('li', class_ = "a-spacing-mini")
        about_count = 0
        if item_a == None:
            about_item = 'N/A'
        else:
            for abouts in item_a:
                about_item = about_item + abouts.text.strip()
                about_count = about_count + 1
                if about_count < len(item_a):
                    about_item = about_item + "; "
        data.append(about_item)''' # TOO LONG TO RUN EACH ITEM
        length = len(df)
        df.loc[length] = data
        
    n = soup.find('a', class_ = "s-pagination-next")
    if n == None:
        dontstop = False
    else:
        nxt = n['href']
        nxt = site + nxt
        url = nxt
        response = requests.get(url)
        soup2 = BeautifulSoup(response.content, 'html.parser')
        soup = BeautifulSoup(soup2.prettify(), 'html.parser')

df

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22679.00,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
4,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
...,...,...,...,...,...
913,40 Shades of Green,1084.80,5.0 out of 5 stars,N/A,https://www.amazon.co.za/40-Shades-Green/dp/B0...
914,Little Red,587.04,4.5 out of 5 stars,N/A,https://www.amazon.co.za/Little-Red-KATY-B/dp/...
915,A Punto Caramelo,667.49,N/A,N/A,https://www.amazon.co.za/Punto-Caramelo-LO-PIB...
916,Music from Marlboro - Busoni: Fantasia Contrap...,509.17,N/A,N/A,https://www.amazon.co.za/Music-Marlboro-Busoni...


In [5]:
url = site + '/s?k=lg&ref=nb_sb_noss' #Site Extension
response = requests.get(url)
soup2 = BeautifulSoup(response.content, 'html.parser')
soup = BeautifulSoup(soup2.prettify(), 'html.parser')
dontstop = True
counter = 0

while dontstop:
    x = soup.find_all('div', class_ = "s-asin")
    count = len(x)
    for i in range(count):
        data = []
    #DATA
        x = soup.find_all('div', class_ = "s-asin")
        items = x[i]
    #NAME
        name = items.find('h2')#.text.strip()
        if name is None:
            name = 'N/A'
        else:
            name = name.text.strip()
        name = str(name)
        if ',' in name:
            name = name.replace(',', ';')
        #print('Name: ', name)
        data.append(name)
    #PRICE
        a = items.find('span', class_ = "a-price")
        '''if a is None:
            a = '0,000.00'
        else:
            a = a.text'''
        if a is None:
            a = items.find('span', class_ = "a-color-base")
            if a is None:
                a = '0,000.00'
            else:
                a = a.text
        else:
            a = a.text
        b = re.findall(r'\d{1,3},?\d{1,3}\.\d{2}', a)
        c = b[0].replace(",", "")
        price = float(c)
        '''if price == 0:
            price = 'N/A'
        else:
            price = "{:.2f}".format(price)'''
        #print('Price: ', price)
        #price = "{:.2f}".format(price)
        data.append(price)
    #RATING
        rating = items.find('span', class_ = "a-icon-alt")#.text.strip()
        if rating is None:
            rating = 'N/A'
        else:
            rating = rating.text.strip()
        rating = str(rating)
        #print('Rating: ', rating)
        for x in rating:
            if x == ',':
                x = ';'
        data.append(rating)
    #RATING_COUNT
        rating_c = items.find('span', class_ = "a-size-base")#.text.strip()
        if rating_c is None:
            rating_c = 'N/A'
        else:
            rating_c = rating_c.text.strip()
            try:
                rating_c = int(rating_c)
            except:
                rating_c = 'N/A'
        rating_c = str(rating_c)
        #print('Rating Count: ', rating_c)
        for x in rating_c:
            if x == ',':
                x = ';'
        data.append(rating_c)
    #URL
        u = items.find('a', href = True)
        if (u == None) or ('#customerReviews' in u) or ('javascript:void' in u):
            item_url = 'N/A'
        else:
            item_url = u['href']
            item_url = site + item_url
        item_url = str(item_url)
        #print('Site: ', item_url)
        for x in item_url:
            if x == ',':
                x = ';'
        data.append(item_url)
        counter = counter + 1
        #print(counter)
        #print('')
    #ABOUT
        '''item_response = requests.get(item_url)
        item_soup2 = BeautifulSoup(item_response.content, 'html.parser')
        item_soup = BeautifulSoup(item_soup2.prettify(), 'html.parser')
        about_item = ""
        item_a = item_soup.find_all('li', class_ = "a-spacing-mini")
        about_count = 0
        if item_a == None:
            about_item = 'N/A'
        else:
            for abouts in item_a:
                about_item = about_item + abouts.text.strip()
                about_count = about_count + 1
                if about_count < len(item_a):
                    about_item = about_item + "; "
        data.append(about_item)''' # TOO LONG TO RUN EACH ITEM
        length = len(df)
        df.loc[length] = data
        
    n = soup.find('a', class_ = "s-pagination-next")
    if n == None:
        dontstop = False
    else:
        nxt = n['href']
        nxt = site + nxt
        url = nxt
        response = requests.get(url)
        soup2 = BeautifulSoup(response.content, 'html.parser')
        soup = BeautifulSoup(soup2.prettify(), 'html.parser')

df

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22679.00,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
4,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
...,...,...,...,...,...
999,RIVERWELD ESAB L-TEC PT-31 JG-40 WSD-LG40 PT-3...,359.80,4.3 out of 5 stars,665,https://www.amazon.co.za/RIVERWELD-PT-31-WSD-L...
1000,Smithsonian Word Search Natural History: Earth...,481.00,4.7 out of 5 stars,N/A,https://www.amazon.co.za/Smithsonian-Word-Sear...
1001,ALANES 30X Zoom Magnifier Clip-On Mobile Phone...,199.00,N/A,N/A,https://www.amazon.co.za/ALANES-Magnifier-Clip...
1002,Finish 24s Auto Dishwasher Tablets; All in 1 M...,159.99,N/A,N/A,https://www.amazon.co.za/Finish-Dishwasher-Tab...


In [6]:
df = df.map(lambda x: np.nan if x in [0, np.nan, 'N/A'] else x)

In [7]:
df

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22679.00,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
4,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
...,...,...,...,...,...
999,RIVERWELD ESAB L-TEC PT-31 JG-40 WSD-LG40 PT-3...,359.80,4.3 out of 5 stars,665,https://www.amazon.co.za/RIVERWELD-PT-31-WSD-L...
1000,Smithsonian Word Search Natural History: Earth...,481.00,4.7 out of 5 stars,NaN,https://www.amazon.co.za/Smithsonian-Word-Sear...
1001,ALANES 30X Zoom Magnifier Clip-On Mobile Phone...,199.00,NaN,NaN,https://www.amazon.co.za/ALANES-Magnifier-Clip...
1002,Finish 24s Auto Dishwasher Tablets; All in 1 M...,159.99,NaN,NaN,https://www.amazon.co.za/Finish-Dishwasher-Tab...


In [8]:
dup = df[df.duplicated(subset = ['URL Link', 'Price', 'Name'])]
dup = dup.sort_values(by = 'URL Link')
dup

,Name,Price,Rating,Rating Count,URL Link
393,65W 3 Port Super Fast Charger with USB & Type-...,227.00,3.0 out of 5 stars,2,https://www.amazon.co.za/Charger-Adapter-Compa...
994,Megadream Android Gamepad Controller; Wireless...,300.00,3.3 out of 5 stars,637,https://www.amazon.co.za/Megadream-Controller-...
475,Penergy QC 3.0 3A Quick Adaptive Fast Charger ...,114.45,NaN,NaN,https://www.amazon.co.za/Penergy-Adaptive-Char...
759,Penergy QC 3.0 3A Quick Adaptive Fast Charger ...,114.45,NaN,NaN,https://www.amazon.co.za/Penergy-Adaptive-Char...
801,Penergy 2.1A Charger with Type C USB Charge Ca...,117.95,NaN,NaN,https://www.amazon.co.za/Penergy-Charger-Charg...
496,2 Pairs Replacement Watch Band Adapter; Watch ...,199.00,NaN,NaN,https://www.amazon.co.za/Replacement-Connector...
965,iOttie Easy One Touch 6 CD Slot Car Phone Moun...,478.55,4.4 out of 5 stars,NaN,https://www.amazon.co.za/iOttie-Easy-Touch-Pho...


In [9]:
#df[df['URL Link'] == dup.iat[7, 4]]

In [10]:
df_dup = df

In [11]:
df_dup = df_dup.drop_duplicates(subset = ['URL Link', 'Price', 'Name'])

In [12]:
df_dup

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22679.00,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
4,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
...,...,...,...,...,...
999,RIVERWELD ESAB L-TEC PT-31 JG-40 WSD-LG40 PT-3...,359.80,4.3 out of 5 stars,665,https://www.amazon.co.za/RIVERWELD-PT-31-WSD-L...
1000,Smithsonian Word Search Natural History: Earth...,481.00,4.7 out of 5 stars,NaN,https://www.amazon.co.za/Smithsonian-Word-Sear...
1001,ALANES 30X Zoom Magnifier Clip-On Mobile Phone...,199.00,NaN,NaN,https://www.amazon.co.za/ALANES-Magnifier-Clip...
1002,Finish 24s Auto Dishwasher Tablets; All in 1 M...,159.99,NaN,NaN,https://www.amazon.co.za/Finish-Dishwasher-Tab...


In [15]:
df_dup.to_csv('C:/UNISA/STA1610/AMAZON_SEARCH_RESULTS.csv', index = False)

In [16]:
#End scrape and clean
#Start read and analyze

In [17]:
import pandas as pd
df2 = pd.read_csv('C:/UNISA/STA1610/AMAZON_SEARCH_RESULTS.csv')
df2
#READ FILE IN

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15.0,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5.0,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3.0,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22679.00,4.1 out of 5 stars,5.0,https://www.amazon.co.za/Samsung-Galaxy-Storag...
4,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9.0,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
...,...,...,...,...,...
992,RIVERWELD ESAB L-TEC PT-31 JG-40 WSD-LG40 PT-3...,359.80,4.3 out of 5 stars,665.0,https://www.amazon.co.za/RIVERWELD-PT-31-WSD-L...
993,Smithsonian Word Search Natural History: Earth...,481.00,4.7 out of 5 stars,NaN,https://www.amazon.co.za/Smithsonian-Word-Sear...
994,ALANES 30X Zoom Magnifier Clip-On Mobile Phone...,199.00,NaN,NaN,https://www.amazon.co.za/ALANES-Magnifier-Clip...
995,Finish 24s Auto Dishwasher Tablets; All in 1 M...,159.99,NaN,NaN,https://www.amazon.co.za/Finish-Dishwasher-Tab...


In [18]:
#Items with ratings but no amount of reviews
df2[(df2['Rating'].isnull() == False) & (df2['Rating Count'].isnull() == True)]

,Name,Price,Rating,Rating Count,URL Link
10,Samsung 60W Type-C to Type-C Cable; 1 Meter Ca...,195.00,4.5 out of 5 stars,NaN,https://www.amazon.co.za/Samsung-Type-C-Cable-...
18,Samsung Galaxy A14 4G 64GB Silver,2899.00,4.4 out of 5 stars,NaN,https://www.amazon.co.za/Samsung-Galaxy-A14-64...
19,Samsung Galaxy A14 4G 64GB black,2929.00,4.4 out of 5 stars,NaN,https://www.amazon.co.za/Samsung-Galaxy-A14-64...
24,Samsung 970 EVO Plus 1TB NVMe Solid State Drive,1749.00,4.8 out of 5 stars,NaN,https://www.amazon.co.za/Samsung-Plus-Solid-St...
26,SAMSUNG Galaxy Watch 6 40mm Bluetooth Smartwat...,4799.00,4.4 out of 5 stars,NaN,https://www.amazon.co.za/SAMSUNG-Bluetooth-Sma...
...,...,...,...,...,...
981,Car Cup Holder Mount Phone Holder with Food Tr...,299.00,4.2 out of 5 stars,NaN,https://www.amazon.co.za/Holder-Compatible-iPh...
985,Essentials Lg Dot Matrix Neverthel,439.36,4.8 out of 5 stars,NaN,https://www.amazon.co.za/Essentials-Lg-Dot-Mat...
987,LG Premium Sketchbook,221.00,4.8 out of 5 stars,NaN,https://www.amazon.co.za/Premium-Sketchbook-Pe...
988,Finish 250ml Auto Dishwasher Machine Cleaner; ...,69.00,4.6 out of 5 stars,NaN,https://www.amazon.co.za/Finish-Dishwasher-Dis...


In [19]:
#declaring items with a review count 
dfR = df2[df2['Rating Count'].isnull() == False]
dfR

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15.0,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5.0,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3.0,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22679.00,4.1 out of 5 stars,5.0,https://www.amazon.co.za/Samsung-Galaxy-Storag...
4,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9.0,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
...,...,...,...,...,...
972,"Kensington Monitor Screen Privacy Filter 23.6""...",3056.06,1.0 out of 5 stars,1.0,https://www.amazon.co.za/Kensington-Monitor-Sc...
976,UPGRADED Lifetime Appliance AGZ72909711 Washpl...,2000.00,4.6 out of 5 stars,709.0,https://www.amazon.co.za/Lifetime-Appliance-AG...
989,Finish 3 Pack Auto Dishwasher Cleaning Pods; I...,99.00,4.5 out of 5 stars,301.0,https://www.amazon.co.za/Finish-Dishwasher-Cle...
990,Finish 80s Auto Dishwasher Tablets; All in 1 M...,279.00,4.4 out of 5 stars,28.0,https://www.amazon.co.za/Finish-Dishwasher-Tab...


In [20]:
#items with more than 50 reviews
dfR[dfR['Rating Count'].astype(int) > 50]

,Name,Price,Rating,Rating Count,URL Link
14,Samsung 25W 10000mAh PD Fast Charging Powerban...,499.00,4.6 out of 5 stars,137.0,https://www.amazon.co.za/Samsung-10000mAh-Char...
16,Samsung 15W 1 Port Fast Travel Adapter with Ty...,399.00,4.6 out of 5 stars,323.0,https://www.amazon.co.za/Samsung-Travel-Adapte...
17,Samsung Galaxy S24 Ultra Standing Grip Case-Da...,629.00,4.4 out of 5 stars,270.0,https://www.amazon.co.za/Samsung-Galaxy-Standi...
20,Samsung EP-DG930IBEGWW USB to Type-C Cable; 1....,199.00,4.3 out of 5 stars,75.0,https://www.amazon.co.za/Samsung-EP-DG930IBEGW...
27,Samsung 25W 1 Port GaN Travel Adapter; White,549.00,4.8 out of 5 stars,373.0,https://www.amazon.co.za/Samsung-Port-Travel-A...
...,...,...,...,...,...
962,OEM 6501ea1001R For LG Washer Water Level Pres...,299.00,4.6 out of 5 stars,60.0,https://www.amazon.co.za/6501ea1001R-Compatibl...
971,Kenu Airframe Ultra with Insta-Grip tech | Uni...,650.82,3.7 out of 5 stars,262.0,https://www.amazon.co.za/Kenu-Airframe-Insta-G...
976,UPGRADED Lifetime Appliance AGZ72909711 Washpl...,2000.00,4.6 out of 5 stars,709.0,https://www.amazon.co.za/Lifetime-Appliance-AG...
989,Finish 3 Pack Auto Dishwasher Cleaning Pods; I...,99.00,4.5 out of 5 stars,301.0,https://www.amazon.co.za/Finish-Dishwasher-Cle...


In [21]:
df2[df2['Price'] > 1000]

,Name,Price,Rating,Rating Count,URL Link
1,SAMSUNG A05 64GB DS BLACK LTE,1799.0,4.0 out of 5 stars,5.0,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.0,5.0 out of 5 stars,3.0,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22679.0,4.1 out of 5 stars,5.0,https://www.amazon.co.za/Samsung-Galaxy-Storag...
5,Samsung Galaxy A35 128GB 5G Dual Sim Smartphon...,5262.0,5.0 out of 5 stars,2.0,https://www.amazon.co.za/Samsung-Galaxy-128GB-...
6,Samsung Galaxy A35 128GB 5G Dual Sim Smartphon...,5399.0,5.0 out of 5 stars,5.0,https://www.amazon.co.za/Samsung-Galaxy-128GB-...
...,...,...,...,...,...
976,UPGRADED Lifetime Appliance AGZ72909711 Washpl...,2000.0,4.6 out of 5 stars,709.0,https://www.amazon.co.za/Lifetime-Appliance-AG...
982,Impact du marketing électronique sur l'intenti...,1338.0,NaN,NaN,https://www.amazon.co.za/marketing-%C3%A9lectr...
983,Impacto do E-Marketing na intenção de compra d...,1283.0,NaN,NaN,https://www.amazon.co.za/Impacto-Marketing-int...
984,Impatto dell'E-Marketing sull'intenzione di ac...,1336.0,NaN,NaN,https://www.amazon.co.za/Impatto-dellE-Marketi...


In [22]:
#Items going for more than 1000 that have more than 50 reviews
dfR[(dfR['Price'] > 1000) & (dfR['Rating Count'].astype(int) > 50)]

,Name,Price,Rating,Rating Count,URL Link
61,SAMSUNG Galaxy A54 256GB 5G Mobile Phone - Awe...,7129.0,4.5 out of 5 stars,511.0,https://www.amazon.co.za/SAMSUNG-Galaxy-256GB-...
73,Samsung Galaxy A05s (4GB; 128GB Storage) | 50 ...,2899.0,4.4 out of 5 stars,265.0,https://www.amazon.co.za/Samsung-Galaxy-Storag...
74,"SAMSUNG Galaxy Tab A9 8.7"" SM-X115 64GB 4GB Fa...",3395.0,4.2 out of 5 stars,81.0,https://www.amazon.co.za/SAMSUNG-SM-X115-Facto...
85,Samsung Galaxy A05s (4GB; 128GB Storage) | 50 ...,2949.0,4.4 out of 5 stars,108.0,https://www.amazon.co.za/Samsung-Galaxy-Storag...
93,Samsung Slim Phone Case with S Pen Holder for ...,1699.0,4.3 out of 5 stars,412.0,https://www.amazon.co.za/Samsung-Phone-Holder-...
...,...,...,...,...,...
926,"LG 22MP410-B Computer Monitor 54.5 cm (21.4"") ...",2257.0,4.3 out of 5 stars,67.0,https://www.amazon.co.za/LG-22MP410-B-Computer...
927,LG 20MK400H-B 19.5 LED Full HD Monitor,1769.0,4.1 out of 5 stars,198.0,https://www.amazon.co.za/LG-20MK400H-B-19-5-Fu...
928,LG 68;6cm/27'' (2560x1440) 27QN880-B 16:9 5ms ...,10731.0,4.5 out of 5 stars,81.0,https://www.amazon.co.za/LG-6cm-2560x1440-27QN...
938,LG Electronics Full HD Monitor 24MP400; 24 Inc...,2804.0,4.3 out of 5 stars,51.0,https://www.amazon.co.za/LG-Electronics-Adjust...


In [25]:
#Average Cost per searched brand
brands = ['Samsung', 'Huawei', 'Sony', 'LG']
SUM = [0.0, 0.0, 0.0, 0.0]
COUNT = [0, 0, 0, 0]
itms = len(df2)
for i in range(itms):
    for x in range(4):
        if brands[x] in df2.at[i, 'Name']:
            SUM[x] = SUM[x] + df2.at[i, 'Price']
            COUNT[x] = COUNT[x] + 1

print("AVERAGE COST FOR EACH BRAND\n")
for x in range(4):
    Avg = SUM[x]/COUNT[x]
    print(brands[x], ": R ", "{:.2f}".format(Avg))

AVERAGE COST FOR EACH BRAND

Samsung : R  2616.93
Huawei : R  733.44
Sony : R  9711.64
LG : R  3362.53
